# Statistical Methods in Image Processing EE-048954
## Homework 1: Kernel Density Estimation and Normalizing Flows
### Due Date: <span style="color:red">May 08, 2022</span>

###  Submission Guidelines

* Submission only in **pairs** on the course website (Moodle).
* Working environment:
    * We encourage you to work in `Jupyter Notebook` online using <a href="https://colab.research.google.com/">Google Colab</a> as it does not require any installation.
* You should submit two **separated** files:
    * A `.ipynb` file, with the name: `ee048954_hw1_id1_id2.ipynb` which contains your code implementations.
    * A `.pdf` file, with the name: `ee048954_hw1_id1_id2.pdf` which is your report containing plots, answers, and discussions.
    * **No handwritten submissions** and no other file-types (`.docx`, `.html`, ...) will be accepted.

### Mounting your drive for saving/loading stuff

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

### Importing relevant libraries for Part I

In [ ]:
## Standard libraries
import os
import math
import time
import numpy as np
import copy
import numpy as np
import numpy.linalg
from scipy.stats import multivariate_normal as mv_normal
from scipy.stats import norm
import pandas as pd
## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
plt.style.use('ggplot')
np.random.seed(0)
from scipy.signal import  convolve2d
import seaborn as sns
from scipy.stats import multivariate_normal
import xarray as xr
from pytictoc import TicToc
import pickle

## Part I: Kernel Density Estimation (30 points)

The multivariate kernel density estimate of a density $f(\mathbf{x})$ given a set of samples $\{\mathbf{x}_i\}$ is given by
$$\hat{f}(\mathbf{x}) = \frac{1}{N} \frac{1}{|H|} \sum_{i=1}^{N}  K(H^{-1} (\mathbf{x}_i-\mathbf{x})),
$$
where $H$ is a bandwidth matrix, $K(\cdot)$ is the kernel and $\{\mathbf{x}_i\}_{i=1}^N$ are $\textit{i.i.d.}$ samples drawn from $f(\mathbf{x})$.

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 1</span>**. Consider the following density functions:
  * Gaussian Mixture:
  $$f(\mathbf{x};\sigma,\{\mu_i\}) = \frac{1}{2\pi\sigma^2} \sum_{m=1}^{M} \frac{1}{M} \exp\left\{-\frac{1}{2\sigma^2}||\mathbf{x}-\mathbf{\mu}_i||^2\right\} ,$$
   with $M = 4$, $\sigma = \frac{1}{2}$, and $\{\mathbf{\mu}_m\} = \{(0,0)^T , (0,2)^T , (2,0)^T , (2,2)^T\}$.
  * Gaussian Mixture with $M = 4$, $\sigma = 1$, and $\{\mathbf{\mu}_m\} = \{(0,0)^T , (0,2)^T , (2,0)^T , (2,2)^T\} $.
  * Spiral with $\theta \sim \mathcal{U}[0,4\pi]$ and $r|\theta \sim \mathcal{N}(\frac{\theta}{2},0.25)$.
  
  For each of the three distributions above, implement a function that draws $N = 10000$ samples ${\mathbf{x}_i}$ from $f(\mathbf{x})$. Display the drawn samples for each distribution separately.

In [ ]:
def set_visualization_settings():
    FIGURE_DPI = 300
    SAVEFIG_DPI = 96
    SMALL_FONT_SIZE = 14
    MEDIUM_FONT_SIZE = 16
    BIG_FONT_SIZE = 18
    TITLE_FONT_SIZE = 18
    SUPTITLE_FONT_SIZE = 22
    COLORS = ["darkblue", "darkgreen", "darkred"]
    plt.rcParams['figure.dpi'] = FIGURE_DPI
    plt.rcParams['savefig.dpi'] = SAVEFIG_DPI


    # Create an array with the colors to use
    # Set a custom color palette
    sns.set_palette(sns.color_palette(COLORS))

    plt.rc('font', size=SMALL_FONT_SIZE)  # controls default text sizes
    plt.rc('axes', titlesize=TITLE_FONT_SIZE)  # fontsize of the axes title
    plt.rc('axes', labelsize=BIG_FONT_SIZE)  # fontsize of the x and y labels
    plt.rc('axes', labelweight='bold')  # weight of the x and y labels
    plt.rc('xtick', labelsize=SMALL_FONT_SIZE)  # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_FONT_SIZE)  # fontsize of the tick labels
    plt.rc('legend', fontsize=MEDIUM_FONT_SIZE)  # legend fontsize
    plt.rc('figure', titlesize=SUPTITLE_FONT_SIZE)  # fontsize of the figure title
    # plt.rc('text', usetex=True)
    plt.rc('font', weight='bold')
    plt.rcParams['text.latex.preamble'] = r'\boldmath'
set_visualization_settings()
def clear_fname(fname):
    return fname.replace('{','').replace('}','').replace('$','').replace('\\','').replace('rm','').replace('^','').replace(',','').replace('_',' ')

def cart2pol(x, y):
    r = np.sqrt(x**2 + y**2)
    theta = np.arctan2(y, x)
    #theta = np.arctan(y, x)
    return(r, theta)

def pol2cart(r, theta):
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    return(x, y)

def sample_from_gaussian_mixture(M,sigma,miu_m,N):
    """
    Sample Gaussian Mixture
    :param M:
    :param sigma:
    :param miu_m:
    :param N:
    :return:
    """
    gauss_components = np.random.choice(M, N ,replace=True)# sample N gaussian components
    xs= np.array([np.random.multivariate_normal(miu,sigma) for miu in miu_m[gauss_components]])
    pdf = 0
    for miu in miu_m:
        pdf+=mv_normal.pdf(xs,miu,sigma)
    samples = pd.DataFrame(xs,columns=['x','y'])
    return samples,pdf/M

def sample_from_spiral_dist(sigma,N):
    p_theta = 1/(4*np.pi)
    tetas = np.random.uniform(low=0,high=4*np.pi,size=N)
    rs = np.array([np.random.normal(teta/2,sigma) for teta in tetas])
    pdf=[]
    for teta,r in zip(tetas,rs):
        p_r_theta = norm.pdf(r,teta/2,sigma)
        p = p_theta *p_r_theta
        pdf.append(p)

    xs = [pol2cart(r,theta) for r,theta in zip(rs,tetas)]
    samples = pd.DataFrame(xs,columns=['x','y'])
    samples_rt = pd.DataFrame([[r,theta] for r,theta in zip(rs,tetas)],columns=['r','theta'])
    return samples,pdf, samples_rt

def draw_samples(samples,pdf_samples,figsize =(12, 9), s_title='samples',
                  elev = 45, azim = 12, label_x ='x',label_y='y',label_z='z',
                 save=False, fig_type='svg'):
    sns.set(style = "darkgrid")
    fig = plt.figure(figsize =figsize)
    ax = plt.axes(projection ='3d')
    values = ax.scatter3D(samples.x,samples.y,pdf_samples,c=pdf_samples,cmap='turbo',s=1)
    clb = fig.colorbar(values, ax = ax,shrink = 0.5, aspect = 5)#, orientation="horizontal")
    clb.ax.set_title(label_z)
    ax.view_init(elev=elev, azim=azim)
    ax.set_xlabel(label_x)
    ax.set_ylabel(label_y)
    ax.zaxis.set_rotate_label(False)
    ax.set_zlabel(label_z,rotation=90)
    plt.tight_layout()
    ax.set_title(s_title)
    if save:
        f_name= clear_fname(s_title)
        fig_path = os.path.join(os.path.curdir, 'figures',f_name+'.'+fig_type)
        print(f"Saving fig to {fig_path}")
        plt.savefig(fig_path)

    # show plot
    plt.show()
    return fig,ax

def kernel(u):
    return np.exp(-u**2/2)/(np.sqrt(2*np.pi))

def plot_density(X,Y,Z,figsize =(15, 9), s_title='Z',
                elev = 45, azim = 12, label_x ='x',label_y='y',label_z='z',
                proj_offset=-0.05, samples=None, save=False, fig_type='svg', alpha=1.0):
    # Creating figure
    sns.set(style = "darkgrid")
    fig = plt.figure(figsize =figsize)
    ax = plt.axes(projection ='3d')

    # Creating plot
    surf = ax.plot_surface(X,Y,Z,
                        cmap = 'turbo',
                        edgecolor ='none',
                        alpha = alpha)
    clb = fig.colorbar(surf, ax = ax,shrink = 0.5, aspect = 5)
    clb.ax.set_title(label_z)
    ax.set_xlabel(label_x)
    ax.set_ylabel(label_y)
    ax.zaxis.set_rotate_label(False)
    ax.set_zlabel(label_z,rotation=90)
    ax.set_title(s_title)
    if samples is not None:
        ax.contourf(X, Y, Z, zdir='z', offset=proj_offset, cmap='turbo')
        # Adjust the limits, ticks and view angle
        ax.set_zlim(proj_offset,Z.max())
        ztick = np.linspace(start=0, stop=density.max()*1.01,num=6).tolist()
        ax.set_zticks(ztick,[f'{value:.2f}' for value in ztick])
        if samples is not None:
            ax.scatter(samples.x,samples.y,proj_offset,s=.05,c='white')
    ax.view_init(elev=elev, azim=azim)
    plt.tight_layout()

    if save:
        f_name= clear_fname(s_title)
        fig_path = os.path.join(os.path.curdir, 'figures',f_name+'.'+fig_type)
        print(f"Saving fig to {fig_path}")
        plt.savefig(fig_path)

    # show plot
    plt.show()
    return fig,ax

def calculate_GMV_pdf(X,Y,mus, sigmas):
    grid = np.dstack((X, Y))
    density = np.zeros(grid.shape[0:2])
    weight = 1/len(mus)
    for mu,signa in zip(mus,sigmas):
        rv = multivariate_normal(mu, sigma)
        z_i = np.reshape(rv.pdf(grid), grid.shape[0:2])
        density += z_i*weight
    return density


In [ ]:
# define grid
xlim = [-10.0,10.0]
ylim = [-10.0,10.0]
nx_grid = 400j
ny_grid = 400j
X, Y = np.mgrid[xlim[0]:xlim[1]:nx_grid, ylim[0]:ylim[1]:ny_grid]


fig_type='svg' # or 'png'
save_fig=False

In [ ]:
# Drawing samples of distributions
N = 10000
samples_1,pdf_1 = sample_from_gaussian_mixture(M=4,sigma= np.eye(2)*0.5,miu_m=np.array([[0,0],[0,2],[2,0],[2,2]]) ,N =N)
samples_2,pdf_2= sample_from_gaussian_mixture(M=4,sigma= np.eye(2)*1. ,miu_m=np.array([[0,0],[0,2],[2,0],[2,2]]) ,N =N)
samples_3,pdf_3,samples_rt = sample_from_spiral_dist(sigma=0.25,N=N)

for ind, (samples,pdf) in enumerate(zip([samples_1,samples_2,samples_3], [pdf_1,pdf_2,pdf_3])):
    name =rf'$f_{ind+1}$'
    draw_samples(samples,pdf,figsize =(10, 8), s_title='samples of '+name,label_z=r'$f(x,y)$',
                 elev = 35, azim = -40,save=save_fig,fig_type=fig_type)

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 2</span>**. Implement a function that accepts samples $\{\mathbf{x}_i\}$ and a bandwidth matrix $ {H}$ and estimates $\hat{f}(\mathbf{x})$ using multivariate kernel density estimation. Use the two-dimension separable kernel $K( {u}) = k(u_1)k(u_2)$ where $k(u)= \frac{1}{\sqrt{2\pi}} \exp \{\frac{-u^2}{2}\}$.

In [ ]:
def KDE(X:np.ndarray,Y:np.ndarray,samples:pd.DataFrame,h:float):
    """
    KDE estimation of density, assuming gaussian kernel
    :param X: meshgrid of X
    :param Y: meshgrid of Y
    :param samples: pd.DataFrame with columns 'x' and 'y' for samples locations
    :param h: Kernel bandwidth (here we assume similar bandwidth for X and Y)
    :return: f_hat - The estimated density based on given samples
    """
    d = float(samples.shape[1]) # d - dimensions of samples
    N = float(samples.shape[0]) # N - number of samples

    # calculate kernel weights using 2D histogram
    d_x = X[1,0] - X[0, 0]
    d_y = Y[0,1] - Y[0, 0]
    edges_x = X[:,0]+.5*d_x
    edges_y = Y[0,:]+.5*d_y
    edge0_x = np.array(X[0, 0]-.5*d_x).reshape(1)
    edge0_y = np.array(Y[0, 0]-.5*d_y).reshape(1)
    edges_x = np.append(edge0_x,edges_x,axis=0)
    edges_y = np.append(edge0_y,edges_y,axis=0)
    kernel_weights, _, _ = np.histogram2d(x=samples.x, y=samples.y,
                                            bins=(edges_x, edges_y),density=False)
    # creating data array of weights, for debugging.
    kernel_weights_da = xr.DataArray(data=kernel_weights,coords={'x':X[:,0],'y':Y[0,:]},attrs={'long_name':r'$N_s$'})

    # setting separable kernel
    kernel_x =kernel(X/h)
    kernel_y =kernel(Y/h)
    K = kernel_x*kernel_y
    norm_factor = (h ** d) * N

    # Calculate KDE using 2D convolution
    f_hat = (1/norm_factor)*convolve2d(K, kernel_weights, mode="same")
    return f_hat, kernel_weights_da

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 3</span>**. For **each** distribution, compare between $f(\mathbf{x})$ and $\hat{f}(\mathbf{x})$ using the bandwidth matrices
${H} = \begin{pmatrix}
	h & 0\\0 & h
	\end{pmatrix}$ with $h = 0.1,0.5,1$. and display the estimation.
	Discuss the trade-off of the choice of $h$.

In [ ]:
LOAD_RESULTS = False  # change to True, if you wish to skip the calculations
KDE_fname= 'results_KDE.pickle'
if LOAD_RESULTS:
    print('Loading pre-calculated KDE results')
    with open(KDE_fname, 'rb') as handle:
        results = pickle.load(handle)
else:
    print('KDE estimation')
    # Estimate KDE for different h sizes
    BW = [.1,.5,1.0]
    results={}
    t = TicToc()
    for indf,(samples) in enumerate([samples_1,samples_2,samples_3]):
        func_name =rf'$f_{indf+1}$'
        results_ind={}
        for indh,(h) in enumerate(BW):
            t.tic()
            f_hat,kernel_weights_da = KDE(X, Y, samples,h)
            t.toc()
            print(f'for estimating f_{indf+1} using h={h}')
            results_ind[rf'h{indh}']=f_hat
        results_ind[rf'hist']=kernel_weights_da
        results[rf'f{indf}'] = results_ind

    # Save results of KDE
    with open(KDE_fname, 'wb') as handle:
        print('Saving KDE results')
        pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Plot the estimated KDE for different h sizes
for indf,(samples) in enumerate([samples_1,samples_2,samples_3]):
    # plot f_hat for each h
    results_ind = results[rf'f{indf}']
    for indh,(h) in enumerate(BW):
        f_hat = results_ind[rf'h{indh}']
        plot_density(X,Y,f_hat,figsize =(10, 8),
                     s_title=fr"$\hat{{f}}_{indf+1}^{{\rm KDE}}\, for\, h={h},\, N={N}$",
                     label_z=r'$\hat{f}(x,y)$',alpha=1.0,
              elev = 35, azim = -40, save=save_fig, fig_type=fig_type)

    # plot kernel weights (Note that these weights are similar for any h. They are related to the grid of X,Y)
    kernel_weights_da = results_ind[rf'hist']
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize =(10,8))
    kernel_weights_da.plot(cmap='turbo',ax=ax)
    func_name =rf'$f_{indf+1}$'
    title_hist = fr'2D histogram of {func_name} samples'
    if save_fig:
        f_name= clear_fname(title_hist)
        fig_path = os.path.join(os.path.curdir, 'figures',f_name+'.'+fig_type)
        print(f"Saving fig to {fig_path}")
        plt.savefig(fig_path)
    ax.set_title(title_hist)
    plt.show()

In [ ]:
# Analyse KDE results of f1
indf=0
# calculating f1 on a dense grid
sigma= np.eye(2)*0.5
sigmas= [sigma,sigma,sigma,sigma]
mus=[[0,0],[0,2],[2,0],[2,2]]
density = calculate_GMV_pdf(X,Y,mus, sigmas)
title = rf'$f_{indf+1}$'
fig, ax =plot_density(X,Y,density,figsize =(10, 8), s_title=title,
                  elev =35, azim = -40,proj_offset=-.03, label_z=r'$f(x,y)$',
                      samples=None,alpha=1.0,
                      save=save_fig, fig_type=fig_type)

# Comparing f1 with the resulted KDE's
results_ind = results[rf'f{indf}']
for indh,(h) in enumerate(BW):
    f_hat = results_ind[rf'h{indh}']
    f_diff = np.sqrt((density-f_hat)**2)
    title = fr"$\Delta f_{indf+1} \, for\, h={h},\, N={N}$"
    fig, ax =plot_density(X,Y,f_diff,figsize =(10, 8), s_title=title,
                  elev =35, azim = -40,proj_offset=-.03, label_z=rf'$\Delta f$',
                      samples=None, alpha=1.0,
                      save=save_fig, fig_type=fig_type)

In [ ]:
# Analyse KDE results of f2
indf=1
# calculating f2 on a dense grid
sigma= np.eye(2)*1
sigmas= [sigma,sigma,sigma,sigma]
mus=[[0,0],[0,2],[2,0],[2,2]]
density = calculate_GMV_pdf(X,Y,mus, sigmas)
title = rf'$f_{indf+1}$'
fig, ax =plot_density(X,Y,density,figsize =(10, 8), s_title=title,
                  elev =35, azim = -40,proj_offset=-.03, label_z=r'$f(x,y)$',
                      samples=None,alpha=1.0,
                      save=save_fig, fig_type=fig_type)

# Comparing f1 with the resulted KDE's
results_ind = results[rf'f{indf}']
for indh,(h) in enumerate(BW):
    f_hat = results_ind[rf'h{indh}']
    f_diff = np.sqrt((density-f_hat)**2)
    title = fr"$\Delta f_{indf+1} \, for\, h={h},\, N={N}$"
    fig, ax =plot_density(X,Y,f_diff,figsize =(10, 8), s_title=title,
                  elev =35, azim = -40,proj_offset=-.03, label_z=rf'$\Delta f$',
                      samples=None, alpha=1.0,
                      save=save_fig, fig_type=fig_type)


In [ ]:
# Analyse KDE results of f3 ?
# indf=2
# # calculating f3 on a dense grid......Not working :-/
# R,THETA = cart2pol(X,Y)
# THETA += np.pi
# SIGMAs = np.ones_like(THETA)*0.25
# THETA_ = 2*R
# MUs = THETA/2
# f_r_theta = np.random.normal(MUs,SIGMAs)
# plt.imshow(f_r_theta)
# #density = calculate_GMV_pdf(X,Y,mus, sigmas)
# title = rf'$f_{indf+1}$'
# fig, ax =plot_density(X,Y,density,figsize =(10, 8), s_title=title,
#                   elev =35, azim = -40,proj_offset=-.03, label_z=r'$f(x,y)$',
#                       samples=None,alpha=1.0,
#                       save=save_fig, fig_type=fig_type)
#
# # Comparing f1 with the resulted KDE's
# results_ind = results[rf'f{indf}']
# for indh,(h) in enumerate(BW):
#     f_hat = results_ind[rf'h{indh}']
#     f_diff = np.sqrt((density-f_hat)**2)
#     title = fr"$\Delta f_{indf+1} \, for\, h={h},\, N={N}$"
#     fig, ax =plot_density(X,Y,f_diff,figsize =(10, 8), s_title=title,
#                   elev =35, azim = -40,proj_offset=-.03, label_z=rf'$\Delta f$',
#                       samples=None, alpha=1.0,
#                       save=save_fig, fig_type=fig_type)

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 4</span>**. Which of the distributions was the easiest/hardest to estimate? Why?

### Importing additional relevant libraries for Part II

In [ ]:
## Scikit-learn built-in dataset generators
from sklearn.datasets import make_moons, make_circles, make_blobs

## Progress bar
import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import Adam
from torch.distributions.multivariate_normal import MultivariateNormal

# Traning will be done on CPU for this homework. 
# For K=4, N=1500, epochs=1000 takes < 3 mins.
device = torch.device("cpu")
print("Using device", device)

## Part II: Invertible Neural Networks (70 points) 

In this part, we will take a closer look at invertible neural networks, otherwise known as *Normalizing Flows*. The most popular, current application of normalizing flows is to model datasets of images. In this part, we will implement a type of flows called *Coupling Flows* for a simplified problem of sampling from toy 2D datasets, although similar concepts (with deeper models + tricks) can be used to model images.

### General Concept 
Recall that given a random vector $Z$ with a density $p_Z(\mathbf{z})$ (e.g. Gaussian) and an invertible function $f$, the density $p_X(\mathbf{x})$ of $X=f(Z)$ is given by:

$$
\log p_X(\mathbf{x}) = \log p_Z(f^{-1}(\mathbf{x})) + \log{} \left|\det \frac{df^{-1}(\mathbf{x})}{d\mathbf{x}}\right|
$$

### Toy Datasets 

The provided function `sample_2d_datasets` samples from 4 different toy datasets that we will use for this part to experiment with NFs. The supported options in this function are `{'Circles', 'Moons', 'GaussiansGrid', 'GaussiansRot'}`, where for the last distribution `'GaussiansRot'`, the function supports a varying number of gaussians using the parameter `num_gaussians`.

In [ ]:
def sample_2d_datasets(dist_type, num_samples=1000, seed=0, num_gaussians=5):
  """
  function samples from simple pre-defined distributions in 2D.
  Inputs:
    - dist_type: str specifying the distribution to be chosen from:
      {'Circles', 'Moons', 'GaussiansGrid', 'GaussiansRot'}
    - num_samples: Number of samples to draw from dist_type (int).
    - seed: Random seed integer.
    - num_gaussians: Number of rotated gaussians if dist_type='GaussiansRot'. 
      (relevant only for dist_type='GaussiansRot', should be a keyword argument)
  Outputs:
    - data (np.array): array of num_samplesx2 samples from dist_type
  """
  np.random.seed(seed)
  if dist_type == 'Circles':
    data = make_circles(num_samples, noise=.1, factor=.8, random_state=seed, shuffle=True)[0]
  elif dist_type == 'Moons':
    data = make_moons(num_samples, noise=.1, random_state=seed, shuffle=True)[0]
  elif dist_type == 'GaussiansGrid':
    centers = np.array([[0,0],[0,2],[2,0],[2,2]])
    data = make_blobs(num_samples, centers=centers, cluster_std=.5, random_state=seed, shuffle=True)[0]
  elif dist_type == 'GaussiansRot':
    angles = np.linspace(0, 2 * np.pi, num_gaussians, endpoint=False)
    centers = np.stack([2.5 * np.array([np.cos(angle), np.sin(angle)]) for angle in angles])
    data = make_blobs(num_samples, centers=centers, cluster_std=np.sqrt(.1), random_state=seed, shuffle=True)[0]
  else:
    raise NotImplementedError
  return data

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 1</span>**. To get acquainted with this function, for each of the 4 distributions above, draw $N = 1000$ samples ${{x}_i}$. Display the drawn samples for each distribution in a separate plot.


In [ ]:
samples = [sample_2d_datasets(dist_type) for dist_type in ['Circles', 'Moons', 'GaussiansGrid', 'GaussiansRot']]

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,4,figsize=(15,3))
for i,sample in enumerate(samples):
    ax[i].scatter(sample[:,0],sample[:,1])

For convience purposes, we wrap the function `sample_2d_datasets` with a `torch.utils.data.Dataset` class, and implement the methods `__len__` and `__getitem__` to sample batches afterward with dataloaders when we train our models.

In [ ]:
class ToyDataset(Dataset):
  def __init__(self, dist_type, num_samples=1000, seed=0, num_gaussians=5):
      """
      Wrapper around the function "sample_2d_datasets" to allow iterating 
      batches using a datalaoder when training our normalizing flow model.
      """
      self.data = sample_2d_datasets(dist_type, num_samples, seed, num_gaussians)
      self.num_samples = num_samples

  def __len__(self):
      return self.num_samples

  def __getitem__(self, index):
      return torch.from_numpy(self.data[index]).type(torch.FloatTensor)

### Coupling Layers 

Next, we look at possible transformations to apply inside the flow, focusing on the simplest and most efficient one. A recent popular flow layer, which works well in combination with deep neural networks, is the coupling layer introduced by [Dinh et al.](https://arxiv.org/abs/1605.08803). The input $\mathbf{x}$ is arbitrarily split into two parts, $\mathbf{x}_{1:j}$ and $\mathbf{x}_{j+1:d}$, of which the first remains unchanged by the flow. Yet, $\mathbf{x}_{1:j}$ is used to parameterize the transformation for the second part, $\mathbf{x}_{j+1:d}$. In this coupling layer, we apply an affine transformation by scaling the input by $\mathbf{s}$ and shifting it by $\mathbf{t}$. In other words, our transformation $\mathbf{y}=f(\mathbf{x})$ looks as follows:

$$\mathbf{y}_{1:j} = \mathbf{x}_{1:j}$$
$$\mathbf{y}_{j+1:d} = \mathbf{s}_{\theta_1}(\mathbf{x}_{1:j}) \odot \mathbf{x}_{j+1:d} + \mathbf{t}_{\theta_2}(\mathbf{x}_{1:j})$$

$\mathbf{y}$ is the output of the flow layer, the functions $\mathbf{s}$ and $\mathbf{t}$ are implemented as neural networks, and the sum and multiplication are performed element-wise. Here's a block diagram that visualize the coupling layer in the form of a computation graph:

<center width="100%"><img src="https://drive.google.com/uc?id=1R_omjSvfBN4xjkJgkmkwPX10jkjz_mmC" width="400px"></center>

For convience, since we train using the log-density, it is common practice to apply an exponential on the predicted scaling factor prior to multiplication with $\mathbf{x}_{j+1:d}$, as this simplifies the calculation of the log-determinant of the Jacobian that we will derive shortly. Hence, the implemented transformation in practice is usually:

$$\mathbf{y}_{1:j} = \mathbf{x}_{1:j}$$
$$\mathbf{y}_{j+1:d} = \exp(\mathbf{s}_{\theta_1}(\mathbf{x}_{1:j})) \odot \mathbf{x}_{j+1:d} + \mathbf{t}_{\theta_2}(\mathbf{x}_{1:j})$$

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 2</span>**. Write down the inverse of this layer $\mathbf{x}=f^{-1}(\mathbf{y})$ for some $\mathbf{x},\mathbf{y} \in \mathbb{R}^d$. Draw the inverse function $f^{-1}(\mathbf{y})$ as a computational graph that has $\mathbf{y}$ as input and $\mathbf{x}$ as output.

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 3</span>**. Write down the Jacobian of this layer $\frac{d \mathbf{y}}{d \mathbf{x}}$. Do you recognize a special structure in this matrix?

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 4</span>**. Write down the explicit expression for the *log*-determinant of the Jacobian matrix from the previous section.

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 5</span>**. Write down the explicit expression for the *log*-determinant of the Jacobian matrix of the *inverse* function $\frac{d \mathbf{x}}{d \mathbf{y}}$.



### Task 2
$$ x_{1:j} = y_{1:j} $$
$$ x_{j+1:d} = (y_{j+1:d}-t_{\theta_2}(y_{1:j})) \odot \exp{(-s_{\theta_1}(y_{1:j}))}$$

### Task 3

\begin{equation*}
J = 
\begin{pmatrix}
I_j & 0 \\
\frac{dy_{j+1:d}}{dx^T_{1:j}} & diag(\exp(s(x_{1:j}))\\
\end{pmatrix}
\end{equation*}

We can notice that this is a lower triangular matrix.

### Task 4

Since the jacobian is a lower triangular matrix the determinant is just the multiplication of the diagonal elements $\exp[\sum_n{s(x_{1:j})_n}] $, so the log-deteminant is simply :
$$\sum_n{s(x_{1:j})_n}$$

### Task 5
According to the inverse function theorem, the matrix inverse of the Jacobian matrix of an invertible function is the Jacobian matrix of the inverse function, so:

$$J_{f^{-1}} = J_{f}^{-1}$$
So the log-determinant would be: 
$$ -\sum_n{s(y_{1:j})_n}$$



In our implementation, we will realize the splitting of variables as masking. The variables to be transformed, $\mathbf{x}_{j+1:d}$, are masked when passing $\mathbf{x}$ to the networks to predict the transformation parameters $\mathbf{s}_{\theta_1}(\mathbf{x}_{1:j})$ and $\mathbf{t}_{\theta_2}(\mathbf{x}_{1:j})$. Also, afterward when applying the transformation (don't forget to exponentiate the scaling!), we mask the parameters for $\mathbf{x}_{1:j}$ so that we have an identity operation for those variables.

For predicting the shifting and scaling parameters for our toy datasets we will be using neural networks with 3 Fully Connected layers with LeakyReLU activations in between. Additionally, for stabilization purposes, we multiply the scaling output $\mathbf{s}_{\theta_1}(\mathbf{x}_{1:j})$ prior to exponentiation with a learnable parameter per dimension `scale_factor` initialized to 0. Meaning, our scaling is initialized to 1 as $\exp(0) = 1$. This prevents sudden large scaling values that can destabilize training (especially in the beginning).

The functions $\mathbf{s}_{\theta_1}(\cdot)$, $\mathbf{t}_{\theta_2}(\cdot)$, and `scale_factor` are already implemented in the provided class `CouplingLayer` below for your convinience.

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 6</span>**. Implement missing `forward` and `inverse` methods in the class `CouplingLayer`:
  * The `forward` method should take in `x` and use the mask `self.mask` and the learnable functions `self.s_func`, `self.scale_factor` and `self.t_func` to predict the transformation parameters and compute `y`. This method should also return the parameter `log_det_jac` which is the log-determinant of the Jacobian.
  * The `inverse` method goes in the other direction. It takes in `y` and uses `self.mask`, `self.s_func`, `self.scale_factor` and `self.t_func` to compute `x`. This method should also return the parameter `inv_log_det_jac` which is the log-determinant of the Jacobian of $f^{-1}(\cdot)$.

In [ ]:
class CouplingLayer(nn.Module):
  def __init__(self, mask):
    super(CouplingLayer, self).__init__()

    # mask for splitting (fixed not learnable)
    self.mask = nn.Parameter(mask, requires_grad=False)

    # scaling function and stabilizing scale_factor init. to 0
    self.s_func = nn.Sequential(nn.Linear(in_features=2, out_features=32), 
                                nn.LeakyReLU(),
                                nn.Linear(in_features=32, out_features=32), 
                                nn.LeakyReLU(),
                                nn.Linear(in_features=32, out_features=2))
    self.scale_factor = nn.Parameter(torch.Tensor(2).fill_(0.0))

    # shifting function
    self.t_func = nn.Sequential(nn.Linear(in_features=2, out_features=32), 
                                nn.LeakyReLU(),
                                nn.Linear(in_features=32, out_features=32), 
                                nn.LeakyReLU(),
                                nn.Linear(in_features=32, out_features=2))

  def forward(self, x):
    """
    TODO: replace y and log_det_jac with your code.
    """
    x_part_1 = torch.mul(x,self.mask)
    inv_mask = 1-self.mask
    x_part_2 = torch.mul(x,inv_mask)
    s = self.s_func(x_part_1)
    t = self.t_func(x_part_1)
    exp_scale = torch.exp(torch.mul(s,self.scale_factor))
    y_part_1 = x_part_1
    y_part_2 = torch.mul(exp_scale,x_part_2) + t
    y = torch.concat((y_part_1[:,torch.where(self.mask==1)],y_part_2[:,torch.where(inv_mask==1)]),dim=1)
    log_det_jac = torch.sum(s)
    return y, log_det_jac

  def inverse(self, y):
    """
    TODO: replace x and inv_log_det_jac with your code.
    """
    y_part_1 = torch.mul(y,self.mask)
    inv_mask = 1-self.mask
    y_part_2 = torch.mul(y,inv_mask)
    s = self.s_func(y_part_1)
    t = self.t_func(y_part_1)
    exp_scale = torch.exp(-torch.mul(s,self.scale_factor))
    x_part_2 = torch.mul((y_part_2 - t),exp_scale)
    x_part_1 = y_part_1

    x = torch.concat((x_part_1[:,torch.where(self.mask==1)],x_part_2[:,torch.where(inv_mask==1)]),dim=1)
    inv_log_det_jac = -torch.sum(s)
    return x, inv_log_det_jac

In [ ]:
# test my implementation
mask = torch.tensor([1.,0.])
l = CouplingLayer(mask)
x = torch.tensor([[1.,2.],[3,4]])
y, jac = l(x)
print('y:' ,y, 'J:', jac)
#test inverse:
x_revert, jac_inv = l.inverse(y)
print('x:' ,x_revert, 'J inv :' ,jac_inv)

assert torch.allclose(x,x_revert), "x input and output aren't the same!"



### Coupling Flows

As you might have guessed by now, a coupling layer is powerful yet still limited in its ability to significantly alter the input. This is because it only operates on a chunk of it with element-wise manipulations due to the invertability constraint. We can go on with making our function $f$ more complex. How can we implement more complex invertible functions? The answer is: invertible function composition. We can stack multiple invertible functions $f_1,\dots,f_K$ (e.g. Coupling Layers) after each other, as all together, they still represent a single, invertible function. Specifically, if $\mathbf{y} = f_1(\mathbf{z})$ and $\mathbf{x} = f_2(\mathbf{y})$ are invertible functions, then $\mathbf{x} = f_2 \circ f_1 (\mathbf{z})$ is an invertible function and its inverse is given by $f_1^{-1} \circ f_2^{-1}$. More importantly, the calculation of the log-determinant of the Jacobian in this case is simple using the chain rule.

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 7</span>**. Assuming $\mathbf{y} = f_1(\mathbf{z})$ and $\mathbf{x} = f_2(\mathbf{y})$ are coupling layers, calculate the log-determinant of the Jacobian $\frac{d \mathbf{x}}{d \mathbf{z}}$. How is it related to the log-determinant of the Jacobians $\frac{d \mathbf{y}}{d \mathbf{z}}$ and $\frac{d \mathbf{x}}{d \mathbf{y}}$?

### Task 7
$$ \log |J_{f_1,f_2}| = \log(|\frac{dx}{dy}\frac{dy}{dz}|)= \log(|\frac{dx}{dy}||\frac{dy}{dz}|)= \log(|\frac{dx}{dy}|+ \log |\frac{dy}{dz}|) = \sum_n{s(y_{1:j})_n} + \sum_n{s(z_{1:j})_n}$$

Coupling layers generalize to any masking technique we could think of. However, the most common approach is to split the input $\mathbf{x}$ in half using the mask. For our toy 2D datasets comprised of samples $\mathbf{x} = (p_x ,p_y) \in \mathbb{R}^{d=2}$, this means that either $\left\{\mathbf{x}_{1:j}=p_x, \mathbf{x}_{j+1:d}=p_y\right\}$, or $\left\{\mathbf{x}_{1:j}=p_y, \mathbf{x}_{j+1:d}=p_x\right\}$. These correspond to masks $\left[1,0\right]^T$ and $\left[0,1\right]^T$ respectively. 
Note that when we apply multiple coupling layers, we invert the masking every other layer so that each variable is transformed a similar amount of times.

#### Intuition in 1D

Intuitively, using multiple, learnable invertible functions, a normalizing flow attempts to transform $p_z(z)$ slowly into a more complex distribution which should finally be $p_x(x)$. We visualize the idea below
(figure credit - [Lilian Weng](https://lilianweng.github.io/lil-log/2018/10/13/flow-based-deep-generative-models.html)):

<center width="100%"><img src="https://lilianweng.github.io/posts/2018-10-13-flow-models/normalizing-flow.png" width="700px"></center>

Starting from $z_0$, which follows the prior Gaussian distribution, we sequentially apply the invertible functions $f_1,f_2,...,f_K$, until $z_K$ represents $x$.

#### Implementation

Using the `CouplingLayer` class from above, here we will implement a class named `CouplingFlow` that is comprised of stacked coupling layers. This class will have the following attributes:
* `num_layers` - a scalar passed at initialization that will determine the number of coupling layers to stack, refered to later as $K$.
* `self.layers` - a Module list comprised of $K$ stacked coupling layers each with its own mask. Note that the masks are fixed and non-learnable therefore we set their `requires_grad` property to `False` inside `CouplingLayer`.
* `self.prior` - This is the prior/base distribution. Here we will use a standard Gaussian distribution with a unit variance per dimension implemented using the `torch.distributions` module.

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 8</span>**. Implement the following 3 methods of this class:
* `log_probability` - method that takes in a batch of samples `x` and returns `log_prob` which is their log-probability $\log p(\mathbf{x})$. For convinience we will assume the overall function $f=f_K\circ f_{K-1} \dots \circ f_1$ satisfies $\mathbf{x} = f(\mathbf{z})$. Hence, to calculate the log-probability you should employ the inverse functions $f_i^{-1}$ starting from the last layer $f_K^{-1}$ (assuming $K$ layers).
* `sample_x` - method that takes in a parameter `num_samples` and returns samples `x` from $p(\mathbf{x})$ alongside their log-probability values `log_prob`. 
* `sample_x_each_step` - method takes in a parameter `num_samples` and returns a list of samples `samples` after each intermediate coupling layer in `self.layers`. The first element of this list is samples from the prior distribution $p(\mathbf{z})$ and the last element is samples from $p(\mathbf{x})$.

In [ ]:
class CouplingFlow(nn.Module):
  def __init__(self, num_layers):
      super(CouplingFlow, self).__init__()

      # concatenate coupling layers with alternating masks
      masks = F.one_hot(torch.tensor([i % 2 for i in range(num_layers)])).float()
      self.layers = nn.ModuleList([CouplingLayer(mask) for mask in masks])

      # define prior distribution to be z~N(0,I)
      self.prior = MultivariateNormal(torch.zeros(2),torch.eye(2))

  def log_probability(self, x):
    """
    TODO: replace log_prob with your code.
    """
    log_det_jac = 0
    num_of_layers = len(self.layers)
    for i, _ in enumerate(self.layers):
        x , log_det_jac_i = self.layers[i](x)
        log_det_jac+=log_det_jac_i
    y=x # switch to y for convinience
    log_p_y= self.prior.log_prob(y)
    log_prob= log_p_y + log_det_jac
    return log_prob

  def sample_x(self, num_samples):
    """
    TODO: replace x and log_prob with your code.
    """
    y = self.prior.sample([num_samples])
    num_of_layers = len(self.layers)
    inv_log_det_jac = 0
    for i, _ in enumerate(self.layers):
        y , inv_log_det_jac_i = self.layers[num_of_layers-i-1].inverse(y)
        inv_log_det_jac+=inv_log_det_jac_i
    x = y
    log_p_y = self.prior.log_prob(y)
    log_prob = log_p_y + inv_log_det_jac # +inv_log_det_jac_i
    return x, log_prob

  def sample_x_each_step(self, num_samples):
    """
    TODO: replace samples with your code.
    """
    log_det_jac = 0
    samples = []
    z = self.prior.sample([num_samples])
    for layer in self.layers:
        z , log_det_jac_i = layer(z)
        samples.append(z)        
    return samples

In [ ]:
# Test my imp.
cf = CouplingFlow(2)
x = torch.tensor([[1,2],[3,4]])
lp = cf.log_probability(x)
print('log(p(x)):',lp.data)


In [ ]:
x,log_prob = cf.sample_x(10)
print('Samples from x (2d): \n',x.data, '\nlog prob:' , log_prob.data)

In [ ]:
samples = cf.sample_x_each_step(2)
print([sample.data for sample in samples])

### Training Coupling Flows

Now that we have finished implementing the flow model, we can start training it. Provided below is an already implemented function `train` to train your `CouplingFlow` models. The function recieves 4 arguments:
* `model` - an instance of the class `CouplingFlow`.
* `data` - an instance of the class `ToyDataset`.
* `epochs` - number of epochs to train the model (int). 
* `batch_size` - the batch size to use in training (int).

In [ ]:
# detach tensor and transfer to numpy
def to_np(x):
  return x.detach().numpy()

# Simple training function
def train(model, data, epochs = 100, batch_size = 64):

  # move model into the device
  model = model.to(device)

  # split into training and validation, and create the loaders
  lengths = [int(len(data)*0.9), len(data) - int(len(data)*0.9)]
  train_set, valid_set = random_split(data, lengths)
  train_loader = DataLoader(train_set, batch_size=batch_size)
  valid_loader = DataLoader(valid_set, batch_size=batch_size)

  # define the optimizer and scheduler
  optimizer = Adam(model.parameters(), lr=1e-3)

  # train the model
  train_losses, valid_losses, min_valid_loss = [], [], np.Inf
  with tqdm.tqdm(range(epochs), unit=' Epoch') as tepoch:
    for epoch in tepoch:

      # training loop
      epoch_loss = 0
      model.train(True)
      for batch_index, training_sample in enumerate(train_loader):
          log_prob = model.log_probability(training_sample)
          loss = - log_prob.mean(0)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          epoch_loss += loss
      epoch_loss /= len(train_loader)
      train_losses.append(np.copy(to_np(epoch_loss)))

      # validation loop
      epoch_loss_valid = 0
      model.train(False)
      for batch_index, valid_sample in enumerate(valid_loader):
        log_prob = model.log_probability(valid_sample)
        loss_valid = - log_prob.mean(0)
        epoch_loss_valid += loss_valid
      
      epoch_loss_valid /= len(valid_loader)
      valid_losses.append(np.copy(to_np(epoch_loss_valid)))
      
      # save best model based off validation loss
      if epoch_loss_valid < min_valid_loss:
        model_best = copy.deepcopy(model)
        min_valid_loss = epoch_loss_valid
        epoch_min = epoch

      # report progress with tqdm pbar
      tepoch.set_postfix(train_loss=to_np(epoch_loss), valid_loss=to_np(epoch_loss_valid))
      
  # report best model on val.
  print('\n Best Model achieved {:.4f} validation loss at epoch {} \n'.
        format(min_valid_loss, epoch_min))

  # if the number of samples is too low take the final weights regardless of 
  # valdiation loss due to weak statistics (overfitting avoided by early stopping)
  if lengths[1] < 500:
    model_best = model

  return model_best, train_losses, valid_losses

Here is an example snippet for using this function to train a flow model with $K=4$ layers on a dataset of $N=1500$ samples $\mathbf{x}_i$ from the `'Moons'` distribution for 1000 epochs:

In [ ]:
# seeds to ensure reproducibility
torch.manual_seed(8)
np.random.seed(0)

# dataset
num_samples = 1500
data = ToyDataset('Moons', num_samples=num_samples)

# learning hyper-parameters
K = 4
nepochs = 1000

# instantiate model and optimize the parameters
Flow_model = CouplingFlow(num_layers=K)
moon_model, train_loss, valid_loss = train(Flow_model, data, epochs=nepochs)

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 9</span>**. For each of the 4 provided distributions (use the default value of `num_gaussians=5` for `'GaussiansRot'`), use a similar snippet to repeat the following:
  * Create a `ToyDataset` instance with $N=1500$ samples from the distribution.
  * Learn a `CouplingFlow` model with $K=4$ layers, by training for 500 epochs. For the `'Moons'` dataset train for 1000 epochs.
  * Plot the estimated density $p(\mathbf{x})$ in $\mathbb{R}^2$. To achieve this, use the method `log_probability` to calculate the log-probability $\log p(\mathbf{x})$ at a pre-determined grid of points $\mathbf{x} \in \left[x_{\min},x_{\max}\right]\times\left[y_{\min},y_{\max}\right]\subset\mathbb{R}^2$ (e.g. using `np.meshgrid`). Sample each coordinate with at least 100 points (i.e. a grid of $100 \times 100$ positions in 2D). Plot the resulting 2D distribution $p(\mathbf{x})=\exp (\log p(\mathbf{x}))$ as an image where the value in each pixel is $p(\mathbf{x})$.
  * Plot samples from intermediate flow layers, including the prior $p(\mathbf{z})$ and the modelled $p(\mathbf{x})$. To achieve this, use the method `sample_x_each_layer` with $N=1000$ samples. Plot the resulting samples from each layer in the **same** axis limits to visualize the transformation of each coupling layer separately. You can use `plt.subplot(..., sharex=True, sharey=True)` to achieve link the axis of all subplots.


Implementation tips:
* Use the same seeds as the example snippet for reproducibility. This will also ensure a non-diverging erroneous behavior with other seeds.
* To make sure the model is not overfitting you can look at the training and the valdiation loss outputs of the provided function `train`. Do not include these in your report, use them just for sanity check.
* Estimate the log-probability in a reasonable vicinity of the training domain. For far away coordinates from the training data, the estimation could be poor locally and might bias the dynamic range of your plot.
* To avoid code duplication, you are encouraged to implement two plotting functions: one for plotting the density, and one for plotting the transformations across layers.

### Analyzing Coupling Layers

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 10</span>**. Train two flow models with a varying number of coupling layers $K=\left\{2, 4\right\}$ for 250 epochs, using $N=1500$ samples from the `'GaussiansRot'` dataset with `num_gaussians=5`. Compare the resulting estimated density $p(\mathbf{x})$ (using a grid of $100 \times 100$ points) and the intermediate distributions of $N=1000$ samples throughout the coupling layers of the model. Which model fits $p(\mathbf{x})$ better? What do you conclude regarding the effect of model depth? explain the result in your report and attach the resulting plots.   

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 11</span>**. Train two flow models with $K=4$ layers for 400 epochs, using a varying number of $N=\left\{1500, 3000\right\}$ samples from the `'GaussiansRot'` dataset with `num_gaussians=5`. Compare the resulting estimated density $p(\mathbf{x})$ (using a grid of $100 \times 100$ points) and the intermediate distributions of $N=1000$ samples throughout the coupling layers of the model. Which model fits $p(\mathbf{x})$ better? What do you conclude regarding the effect of training set size? explain the result in your report and attach the resulting plots.

<img src="https://img.icons8.com/offices/80/000000/making-notes.png" style="height:30px;display:inline\">**<span style="color:red">Task 12</span>**. Train two flow models with $K=4$ layers for 250 epochs, using $N=1500$ samples from the `'GaussiansRot'` dataset with a varying number of Gaussians `num_gaussians`$=\left\{3, 7\right\}$. Compare the resulting estimated density $p(\mathbf{x})$ (using a grid of $100 \times 100$ points) and the intermediate distributions of $N=1000$ samples throughout the coupling layers of the model. Which distribution $p(\mathbf{x})$ is fitted better by the model? What do you conclude regarding the effect of data complexity? explain the result in your report and attach the resulting plots.

### Conclusion

In conclusion, we have seen how to implement our own normalizing flow on toy 2D datasets. However, as mentioned in the beginning of Part II, similar models with significantly more layers and additional tricks can be used to model images (e.g. [Glow](http://papers.nips.cc/paper/8224-glow-generative-flow-with-invertible-1x1-convolutions.pdf)). The most common flow element, the coupling layer, is simple to implement, and yet effective. Normalizing flows are an interesting generative model compared to GANs, as they allow an exact likelihood estimate in continuous space, and we have the guarantee that every possible input $x$ has a corresponding latent vector $z$. Recent advances in [Neural ODEs](https://arxiv.org/pdf/1806.07366.pdf) allow a flow with infinite number of layers, called Continuous Normalizing Flows, whose potential is yet to fully explore.

## References and Credits

[1] Dinh, L., Sohl-Dickstein, J., and Bengio, S. (2017). “Density estimation using Real NVP,” In: 5th International Conference on Learning Representations, ICLR 2017. [Link](https://arxiv.org/abs/1605.08803)

[2] Kingma, D. P., and Dhariwal, P. (2018). “Glow: Generative Flow with Invertible 1x1 Convolutions,” In: Advances in Neural Information Processing Systems, vol. 31, pp. 10215--10224. [Link](http://papers.nips.cc/paper/8224-glow-generative-flow-with-invertible-1x1-convolutions.pdf)

[3] University of Amsterdam, Deep Learning 1, Tutorial 11. [Link](https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial11/NF_image_modeling.html#Normalizing-Flows-as-generative-model)

[4] Technical University of Munich, Machine Learning for Graphs and Sequential Data, Generative Models. [Link](https://www.in.tum.de/en/daml/teaching/summer-term-2020/machine-learning-for-graphs-and-sequential-data/)